In [ ]:
import pandas as pd
from pathlib import Path
import os
import numpy as np
from collections import defaultdict
import requests_cache
from fp.fp import FreeProxy
from retry_requests import retry
import openmeteo_requests
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import matplotlib.dates as mdates
from sklearn.preprocessing import StandardScaler
import joblib


In [ ]:
DATA_DIR = Path("data") 
REG_ELCTR_DIR = DATA_DIR / "region_electricity_data"
OUTPUT_FILE = DATA_DIR / "combined_region_electricity.csv"
OUTPUT_PREPARED_FILE = DATA_DIR / "ready2use_region_electricity.csv"
os.makedirs(DATA_DIR, exist_ok=True)

if OUTPUT_FILE.exists():
    print("Output file already exists. Exiting.")
    df = pd.read_csv(OUTPUT_FILE)
else:
    dfs = []

    for file in REG_ELCTR_DIR.iterdir():
        if file.suffix in [".xls", ".xlsx"]:
            df = pd.read_excel(file)
            dfs.append(df)

    print(f"Loaded {len(dfs)} files")


In [ ]:
if not OUTPUT_FILE.exists():
    df = pd.concat(dfs, ignore_index=True)
    print("Merged shape:", df.shape)
    df.head(2)

In [ ]:
if not OUTPUT_FILE.exists():
    KEEP_COLS = ['Region', 'UTC time','Demand','Net generation']

    to_drop = [c for c in df.columns if c not in KEEP_COLS]
    df = df.drop(columns=to_drop)
    df["UTC time"] = pd.to_datetime(df["UTC time"], utc=True, errors="coerce")
    df = df.rename(columns={"UTC time": "timestamp_utc"})
    df = df.sort_values(["Region", "timestamp_utc"]).reset_index(drop=True)
    df = df[df["timestamp_utc"] >= "2022-01-01"].copy()
    df = df[df["timestamp_utc"] <= "2025-10-12"].copy()
    print(df.isna().sum())
    print("Duplicates:", df.duplicated(["Region", "timestamp_utc"]).sum())
    print(df.head())

* **Region** - Код енергетичного регіону США (EIA balancing / monitoring region).
* **Demand** - Фактичне споживання електроенергії в регіоні у цей момент часу, MWh
* **Net generation** - Чиста генерація в регіоні: Net generation = local generation - internal consumption of power plants

In [ ]:
if not OUTPUT_FILE.exists():
    print("Rows with NaNs (before imputation):")
    print(df[df[["Demand", "Net generation"]].isna().any(axis=1)].head(20))
    print("\nNaN counts before:")
    print(df[["Demand", "Net generation"]].isna().sum())
    # допоміжні колонки
    df["date"] = df["timestamp_utc"].dt.date
    # імпутація по (Region, date)
    for col in ["Demand", "Net generation"]:
        df[col] = df[col].fillna(
            df.groupby(["Region", "date"])[col].transform("median")
        )
    # fallback — по регіону
    for col in ["Demand", "Net generation"]:
        df[col] = df[col].fillna(
            df.groupby("Region")[col].transform("median")
        )
    # перевірка
    print(df[["Demand", "Net generation"]].isna().sum())
    df.to_csv(OUTPUT_FILE, index=False)
else:
    df = pd.read_csv(OUTPUT_FILE)

In [ ]:
US_REGION_COORDS = {
    "TEX":  (31.0, -99.0),   # Texas
    "CAL":  (36.5, -119.5),  # California
    "MIDW": (41.8, -93.0),   # Midwest (Iowa-ish)

    "CENT": (39.0, -98.0),   # Central US (Kansas)
    "SE":   (33.0, -84.0),   # Southeast (Georgia)
    "SW":   (34.0, -111.0),  # Southwest (Arizona)
    "MIDA": (39.0, -77.0),   # Mid-Atlantic (Maryland)

    "NE":   (44.0, -71.5),   # New England
    "NY":   (43.0, -75.0),   # New York
    "NW":   (47.5, -120.5),  # Northwest (Washington)
    "TEN":  (35.8, -86.4),   # Tennessee
    "FLA":  (28.0, -82.0),   # Florida
    "CAR":  (35.5, -79.0),   # Carolinas
}

CACHE_FILE = DATA_DIR / "us_weather.cache_openmeteo"
US_WEATHER_CHECKPOINT = DATA_DIR / "us_weather_checkpoint.csv"

# ============================================================
# FAST PATH: if checkpoint exists > load and exit
# ============================================================
if US_WEATHER_CHECKPOINT.exists():
    print("[INFO] Weather checkpoint found, loading CSV.")
    us_weather_df = pd.read_csv(
        US_WEATHER_CHECKPOINT,
        parse_dates=["timestamp_utc"]
    ).sort_values(["us_region", "timestamp_utc"]).reset_index(drop=True)

    us_weather_df.head()
else:
    # ========================================================
    # Proxy generator
    # ========================================================
    def proxy_generator():
        while True:
            try:
                yield FreeProxy(rand=True, timeout=2).get()
            except Exception:
                time.sleep(1)

    proxy_pool = proxy_generator()

    # ========================================================
    # Open-Meteo client factory
    # ========================================================
    def get_openmeteo_client(proxy=None):
        session = requests_cache.CachedSession(CACHE_FILE, expire_after=-1)
        if proxy:
            session.proxies = {"http": proxy, "https": proxy}
        session = retry(session, retries=3, backoff_factor=0.3)
        return openmeteo_requests.Client(session=session)

    # ========================================================
    # Weather fetch (with proxy + fallback)
    # ========================================================
    def fetch_weather_hourly(lats, lons, start_date, end_date, max_attempts=10):
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": lats,
            "longitude": lons,
            "start_date": start_date,
            "end_date": end_date,
            "hourly": [
                "temperature_2m",
                "relative_humidity_2m",
                "precipitation",
                "snowfall",
                "cloud_cover",
                "wind_speed_10m",
                "wind_gusts_10m",
                "surface_pressure",
            ],
            "timezone": "UTC",
        }

        # --- try with proxies
        for _ in range(max_attempts):
            proxy = next(proxy_pool)
            try:
                client = get_openmeteo_client(proxy)
                return client.weather_api(url, params=params)
            except Exception:
                time.sleep(2)

        # --- fallback: direct (no proxy)
        try:
            print("[WARN] Proxy attempts failed, retrying without proxy")
            client = get_openmeteo_client(proxy=None)
            return client.weather_api(url, params=params)
        except Exception as e:
            raise RuntimeError(f"Weather fetch failed completely: {e}")

    # ========================================================
    # Time range
    # ========================================================
    START_DATE = "2022-01-01"
    END_DATE   = "2025-10-12"

    # ========================================================
    # Batch download
    # ========================================================
    weather_frames = []
    BATCH_SIZE = 4  # менше — стабільніше
    regions = list(US_REGION_COORDS.keys())

    for i in range(0, len(regions), BATCH_SIZE):
        batch = regions[i:i + BATCH_SIZE]
        lats, lons = zip(*(US_REGION_COORDS[r] for r in batch))

        print(f"[INFO] Fetching {batch}")

        responses = fetch_weather_hourly(
            list(lats), list(lons), START_DATE, END_DATE
        )

        for j, resp in enumerate(responses):
            region = batch[j]
            hourly = resp.Hourly()

            times = pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s"),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s"),
                freq="H",
                inclusive="left",
            )

            wdf = pd.DataFrame({
                "timestamp_utc": times,
                "us_region": region,
                "temperature": hourly.Variables(0).ValuesAsNumpy(),
                "humidity": hourly.Variables(1).ValuesAsNumpy(),
                "precipitation": hourly.Variables(2).ValuesAsNumpy(),
                "snowfall": hourly.Variables(3).ValuesAsNumpy(),
                "cloud_cover": hourly.Variables(4).ValuesAsNumpy(),
                "wind_speed": hourly.Variables(5).ValuesAsNumpy(),
                "wind_gusts": hourly.Variables(6).ValuesAsNumpy(),
                "surface_pressure": hourly.Variables(7).ValuesAsNumpy(),
            })

            weather_frames.append(wdf)

        time.sleep(2)

    # ========================================================
    # Final dataset + save
    # ========================================================
    us_weather_df = (
        pd.concat(weather_frames, ignore_index=True)
        .drop_duplicates(["timestamp_utc", "us_region"])
        .sort_values(["us_region", "timestamp_utc"])
        .reset_index(drop=True)
    )

    us_weather_df["timestamp_utc"] = pd.to_datetime(us_weather_df["timestamp_utc"],
                                                    utc=True,
                                                    errors="coerce")

    us_weather_df.to_csv(US_WEATHER_CHECKPOINT, index=False)
    print("[OK] Weather data downloaded and saved")

us_weather_df["timestamp_utc"] = pd.to_datetime(us_weather_df["timestamp_utc"],
                                                    utc=True,
                                                    errors="coerce")
us_weather_df.head()


In [ ]:
df["timestamp_utc"] = pd.to_datetime(df["timestamp_utc"],
                                     utc=True,
                                     errors="coerce")
df_us = df.merge(us_weather_df,
    left_on=["Region", "timestamp_utc"],
    right_on=["us_region", "timestamp_utc"],
    how="left"
)

df_us = df_us.drop(columns=["us_region"])
df_us.head()

Мапимо цих 14 регіонів до наших 27, шляхом many-to-many, де один великий штат мапиться на декілька Укр. регіонів тощо:

In [ ]:
CANONICAL_REGIONS = [ # P.S. розмічено чатом гпт =)
    # name, category, scale
    ("Вінницька",        "MID",   0.65),
    ("Волинська",        "SMALL", 0.35),
    ("Дніпропетровська", "BIG",   1.10),
    ("Донецька",         "BIG",   1.00),
    ("Житомирська",      "MID",   0.50),
    ("Закарпатська",     "SMALL", 0.30),
    ("Запорізька",       "BIG",   0.95),
    ("Івано-Франківська","SMALL", 0.35),
    ("Київська",         "BIG",   0.90),
    ("Кіровоградська",   "MID",   0.45),
    ("Луганська",        "MID",   0.40),
    ("Львівська",        "SMALL", 0.70),
    ("Миколаївська",     "MID",   0.60),
    ("Одеська",          "BIG",   1.00),
    ("Полтавська",       "MID",   0.70),
    ("Рівненська",       "SMALL", 0.35),
    ("Сумська",          "SMALL", 0.30),
    ("Тернопільська",    "SMALL", 0.30),
    ("Харківська",       "BIG",   1.05),
    ("Херсонська",       "MID",   0.45),
    ("Хмельницька",      "MID",   0.55),
    ("Черкаська",        "MID",   0.60),
    ("Чернівецька",      "SMALL", 0.25),
    ("Чернігівська",     "SMALL", 0.30),
    ("Київ",             "BIG",   1.20),
    ("м. Севастополь",   "SMALL", 0.15),
    ("АР Крим",          "SMALL", 0.20),
]


REGION_MAP = {
    "Дніпропетровська область": "Дніпропетровська",
    "Харківська область": "Харківська",
    "Полтавська область": "Полтавська",
    "Запорізька область": "Запорізька",
    "Сумська область": "Сумська",
    "Одеська область": "Одеська",
    "Кіровоградська область": "Кіровоградська",
    "Миколаївська область": "Миколаївська",
    "м. Київ": "Київ",
    "Черкаська область": "Черкаська",
    "Львівська область": "Львівська",
    "Хмельницька область": "Хмельницька",
    "Волинська область": "Волинська",
    "Івано-Франківська область": "Івано-Франківська",
    "Закарпатська область": "Закарпатська",
    "Рівненська область": "Рівненська",
    "Тернопільська область": "Тернопільська",
    "Чернівецька область": "Чернівецька",
    "Херсонська область": "Херсонська",
    "Вінницька область": "Вінницька",
    "Житомирська область": "Житомирська",
    "Київська область": "Київська",
    "Чернігівська область": "Чернігівська",
    "Луганська область": "Луганська",
    "Автономна Республіка Крим": "АР Крим",
    "м. Севастополь": "м. Севастополь",
}


In [ ]:
UA_SCALE = {}
UA_CATEGORY = {}
UA_BY_CATEGORY = defaultdict(list)

for name, category, scale in CANONICAL_REGIONS:
    UA_SCALE[name] = scale
    UA_CATEGORY[name] = category
    UA_BY_CATEGORY[category].append(name)

US_GROUPS = {
    "BIG":   ["TEX", "CAL", "MIDW"],
    "MID":   ["CENT", "SE", "SW", "MIDA"],
    "SMALL": ["NE", "NY", "NW", "TEN", "FLA", "CAR"],
}

US_TO_UA_MAP = {
    us_region: UA_BY_CATEGORY[category]
    for category, us_regions in US_GROUPS.items()
    for us_region in us_regions
}


In [ ]:
print(US_TO_UA_MAP["TEX"])
WEATHER_COLS = [
    "temperature",
    "humidity",
    "precipitation",
    "snowfall",
    "cloud_cover",
    "wind_speed",
    "wind_gusts",
    "surface_pressure",
]
#один рядок US - кілька рядків UA, але load масштабується по к-сть регіонів та UA_SCALE
def expand_us_to_ua(df):
    rows = []

    for _, r in df.iterrows():
        ua_regions = US_TO_UA_MAP[r["Region"]]
        base_share = 1.0 / len(ua_regions)

        for ua in ua_regions:
            scale = UA_SCALE[ua]

            row = {
                "timestamp_utc": r["timestamp_utc"],
                "us_region": r["Region"],
                "ua_region": ua,

                # load
                "demand_ua": r["Demand"] * base_share * scale,
                "generation_ua": r["Net generation"] * base_share * scale,
            }

            # weather (без масштабування)
            for c in WEATHER_COLS:
                row[c] = r[c]

            rows.append(row)

    return pd.DataFrame(rows)

ua_df = expand_us_to_ua(df_us)
ua_df = ua_df.sort_values(["ua_region", "timestamp_utc"]).reset_index(drop=True)
ua_df.head()

Додаємо тривоги (+-15хв погодинне розбиття займає)

In [ ]:
ALERTS_CSV = DATA_DIR / "alerts_merged_sorted.csv"
ALERTS_HOURLY_CSV = DATA_DIR / "alerts_hourly_merged_sorted.csv"

if ALERTS_HOURLY_CSV.exists():
    print("[INFO] Alerts hourly CSV found, loading.")
    alerts_hourly = pd.read_csv(
        ALERTS_HOURLY_CSV,
        parse_dates=["timestamp_utc"]
    ).sort_values(["ua_region", "timestamp_utc"]).reset_index(drop=True)
else:
    # ============================================================
    # Load alerts
    # ============================================================
    alerts_df = pd.read_csv(
        ALERTS_CSV,
        parse_dates=["Оголошено о", "Закінчено о"]
    )

    # ============================================================
    # Normalize timestamps (UTC)
    # ============================================================
    alerts_df["start_ts"] = pd.to_datetime(alerts_df["Оголошено о"], utc=True)
    alerts_df["end_ts"] = pd.to_datetime(
        alerts_df["Закінчено о"], utc=True, errors="coerce"
    )

    # fallback: якщо немає кінця + 1 година
    alerts_df["end_ts"] = alerts_df["end_ts"].fillna(
        alerts_df["start_ts"] + pd.Timedelta(hours=1)
    )

    # ============================================================
    # Damage flag
    # ============================================================
    alerts_df["isDamaged"] = (
        alerts_df["Повідомлення"].notna() &
        (alerts_df["Повідомлення"].astype(str).str.strip() != "")
    )

    # ============================================================
    # Normalize regions → canonical UA
    # ============================================================
    alerts_df["ua_region"] = alerts_df["Регіон"].map(REGION_MAP)

    CANONICAL_REGION_NAMES = {name for name, _, _ in CANONICAL_REGIONS}
    alerts_df = alerts_df[alerts_df["ua_region"].isin(CANONICAL_REGION_NAMES)]

    # ============================================================
    # global hourly grid
    # ============================================================
    GLOBAL_START = alerts_df["start_ts"].min().floor("h")
    GLOBAL_END   = alerts_df["end_ts"].max().ceil("h")

    ALL_HOURS = pd.date_range(
        start=GLOBAL_START,
        end=GLOBAL_END,
        freq="h",
        inclusive="left"
    )

    # ============================================================
    # Expand alerts into hourly
    # ============================================================
    alert_rows = []

    for _, r in alerts_df.iterrows():
        start = r["start_ts"].floor("h")
        end   = r["end_ts"].ceil("h")

        hours = pd.date_range(
            start=start,
            end=end,
            freq="h",
            inclusive="left"
        )

        for ts in hours:
            alert_rows.append({
                "timestamp_utc": ts,
                "ua_region": r["ua_region"],
                "alert_active": 1,
                "isDamaged": r["isDamaged"],
            })

    alerts_active = pd.DataFrame(alert_rows)

    # ============================================================
    # Aggregate overlaps
    # ============================================================
    alerts_active = (
        alerts_active
        .groupby(["timestamp_utc", "ua_region"], as_index=False)
        .agg(
            alert_active=("alert_active", "max"),
            isDamaged=("isDamaged", "any"),
        )
    )

    # ============================================================
    # FULL hourly grid: region x hour 
    # ============================================================
    full_index = pd.MultiIndex.from_product(
        [CANONICAL_REGION_NAMES, ALL_HOURS],
        names=["ua_region", "timestamp_utc"]
    )

    alerts_hourly = (
        pd.DataFrame(index=full_index)
        .reset_index()
        .merge(
            alerts_active,
            on=["ua_region", "timestamp_utc"],
            how="left"
        )
    )

    # ============================================================
    # Fill NO-ALERT hours with zeros
    # ============================================================
    alerts_hourly["alert_active"] = alerts_hourly["alert_active"].fillna(0).astype(int)
    alerts_hourly["isDamaged"] = alerts_hourly["isDamaged"].fillna(False)

    # ============================================================
    # Final sanity
    # ============================================================
    alerts_hourly = alerts_hourly.sort_values(
        ["ua_region", "timestamp_utc"]
    ).reset_index(drop=True)

    alerts_hourly.to_csv(ALERTS_HOURLY_CSV, index=False)
    
alerts_hourly.head(10)


накладаємо тривоги на всі регіони та дати

In [ ]:
ua_df["timestamp_utc"] = pd.to_datetime(ua_df["timestamp_utc"], utc=True)
alerts_hourly["timestamp_utc"] = pd.to_datetime(alerts_hourly["timestamp_utc"], utc=True)
ua_df_merged = (ua_df.merge(alerts_hourly, on=["ua_region", "timestamp_utc"], how="left"))
ua_df_merged["alert_active"] = (ua_df_merged["alert_active"].fillna(0).astype(int))
ua_df_merged["isDamaged"] = (ua_df_merged["isDamaged"].fillna(False))
ua_df_merged.head()


In [ ]:
print("Total rows:", len(ua_df_merged))

print("\nAlert distribution:")
print(ua_df_merged["alert_active"].value_counts())

print("\nDamage distribution:")
print(ua_df_merged["isDamaged"].value_counts())

invalid = ua_df_merged.query("isDamaged == True and alert_active == 0")
print("\nInvalid damage rows:", len(invalid))


Тривог по регіонам:

In [ ]:
alerts_by_region = (
    ua_df_merged
    .groupby("ua_region")["alert_active"]
    .sum()
    .rename("alert_hours")
    .sort_values(ascending=False)
)

alerts_by_region.head(10)

Пошкоджень по регіонам

In [ ]:
damage_by_region = (
    ua_df_merged
    .groupby("ua_region")["isDamaged"]
    .sum()
    .rename("damage_hours")
    .sort_values(ascending=False)
)

damage_by_region.head(10)

Пошкодження коли були тривоги

In [ ]:
p_damage_given_alert = (
    ua_df_merged
    .query("alert_active == 1")
    .groupby("ua_region")["isDamaged"]
    .mean()
    .rename("P(damage | alert)")
    .sort_values(ascending=False)
)

p_damage_given_alert.head(15)

Дифіцити енергії

In [ ]:
ua_df_merged["supply_ok"] = ua_df_merged["generation_ua"] >= ua_df_merged["demand_ua"]
total_rows = len(ua_df_merged)
ok_rows = ua_df_merged["supply_ok"].sum()
bad_rows = total_rows - ok_rows

print(f"Total rows: {total_rows:,}")
print(f"Supply OK: {ok_rows:,} ({ok_rows/total_rows:.2%})")
print(f"Supply deficit: {bad_rows:,} ({bad_rows/total_rows:.2%})")


In [ ]:
ua_df_merged["energy_deficit"] = (ua_df_merged["demand_ua"] - ua_df_merged["generation_ua"])
ua_df_merged.head()

Після накладання тривог потрібно логічним чином зімітувати їх вплив на на систему:

In [ ]:
ALERT_GEN_DERATE = 0.01 # припустимо втрати генерації під час тривоги = 1%
ALERT_DEMAND_NOISE = 0.05 # +/- 5% шум попиту під час тривоги

DAMAGE_GEN_DERATE_MIN = 0.35 # мінімальна втрата при пошкодженні
DAMAGE_GEN_DERATE_MAX = 0.55 # максимальна
DAMAGE_TAU_RECOVERY = 36 # годин на відновлення

In [ ]:
# попит стає менш передбачуваним під час тривоги
rng = np.random.default_rng(42)
ua_df_merged["demand_ua_adj"] = ua_df_merged["demand_ua"]

alert_mask = ua_df_merged["alert_active"] == 1

ua_df_merged.loc[alert_mask, "demand_ua_adj"] *= (
    1.0 + rng.normal(loc=0.0, scale=ALERT_DEMAND_NOISE, size=alert_mask.sum())
)

ua_df_merged["generation_ua_adj"] = ua_df_merged["generation_ua"]
ua_df_merged.loc[alert_mask, "generation_ua_adj"] *= (1.0 - ALERT_GEN_DERATE)


In [ ]:
# damage_event = True -> початок деградації
ua_df_merged["damage_event"] = ((ua_df_merged["isDamaged"] == True) &
    (ua_df_merged.groupby("ua_region")["isDamaged"].shift(1) == False)
)
ua_df_merged["damage_derate"] = 0.0
for region, g in ua_df_merged.groupby("ua_region"):
    idx = g.index
    damage_start_times = g.loc[g["damage_event"], "timestamp_utc"]

    for t0 in damage_start_times:
        mask = (
            (ua_df_merged["ua_region"] == region) &
            (ua_df_merged["timestamp_utc"] >= t0))

        dt = (ua_df_merged.loc[mask, "timestamp_utc"] - t0).dt.total_seconds() / 3600.0

        severity = rng.uniform(DAMAGE_GEN_DERATE_MIN,
                               DAMAGE_GEN_DERATE_MAX)

        ua_df_merged.loc[mask, "damage_derate"] += (
            severity * np.exp(-dt / DAMAGE_TAU_RECOVERY)) # експоненціальне відновлення

ua_df_merged["generation_ua_adj"] = ua_df_merged["generation_ua"]

ua_df_merged["generation_ua_adj"] *= (1.0 - ua_df_merged["damage_derate"].clip(upper=0.9))
# генерація падає різко, відновлюється поступово


In [ ]:
ua_df_merged["energy_deficit_adj"] = (
    ua_df_merged["demand_ua_adj"] -
    ua_df_merged["generation_ua_adj"]
).clip(lower=0)

ua_df_merged["supply_ok_adj"] = (
    ua_df_merged["generation_ua_adj"] >=
    ua_df_merged["demand_ua_adj"]
)

ua_df_merged.to_csv(OUTPUT_PREPARED_FILE, index=False)
ua_df_merged.tail(3)

In [ ]:
REGION = "Київ"

ua_df_merged["date"] = ua_df_merged["timestamp_utc"].dt.date

damage_days = (
    ua_df_merged.query("ua_region == @REGION")
      .groupby("date")["isDamaged"]
      .any()
)

damage_day = damage_days[damage_days].index.min()
print("Damage day:", damage_day)

start = pd.to_datetime(damage_day).tz_localize("UTC") - pd.Timedelta(days=1)
end   = pd.to_datetime(damage_day).tz_localize("UTC") + pd.Timedelta(days=3)

dk = ua_df_merged[
    (ua_df_merged["ua_region"] == REGION) &
    (ua_df_merged["timestamp_utc"] >= start) &
    (ua_df_merged["timestamp_utc"] < end)
].sort_values("timestamp_utc")

fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(dk["timestamp_utc"], dk["demand_ua_adj"],
        label="Demand (MW)", linewidth=2)

ax.plot(dk["timestamp_utc"], dk["generation_ua_adj"],
        label="Generation (MW)", linewidth=2)

damage_ts = pd.to_datetime(damage_day).tz_localize("UTC")
ax.axvline(damage_ts, color="red", linestyle="--", linewidth=2,
           label="Damage event")

ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%m %H:%M"))

ax.set_title(
    f"{REGION}: Demand vs Generation\n"
    "1 day before and 2 days after damage event",
    fontsize=14
)

ax.set_xlabel("Time (UTC)")
ax.set_ylabel("Power (MW)")
ax.grid(True, alpha=0.3)
ax.legend()

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Як можна побачити попит почав змінюватися після дамагу, генерація сильно змінюватися, але із поступовим відновленням протягом 36 годин. Аналогічна картина із дефіцитом

In [ ]:
plt.figure(figsize=(14, 4))
plt.plot(dk["timestamp_utc"], dk["energy_deficit_adj"], color="darkred")
plt.axhline(0, linestyle="--", color="black", alpha=0.6)
plt.axvline(damage_ts, color="red", linestyle="--")

plt.title(f"{REGION}: Energy deficit (Demand - Generation)")
plt.xlabel("Time (UTC)")
plt.ylabel("Deficit (MW)")
plt.grid(alpha=0.3)

plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=2))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%d-%m %H:%M"))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


Погода показує логічні зміни в попиті

In [ ]:
REGION = "Київ"
df = ua_df_merged.query("ua_region == @REGION").copy()

fig, axes = plt.subplots(2, 4, figsize=(20, 8), sharey=False)
fig.suptitle(f"{REGION}: Weather impact on Demand, Generation and Damage", fontsize=16)

# -------------------------
# Row 1 — Demand
# -------------------------
axes[0, 0].scatter(df["temperature"], df["demand_ua_adj"], alpha=0.15, s=10)
axes[0, 0].set_title("Temperature → Demand")
axes[0, 0].set_xlabel("°C")
axes[0, 0].set_ylabel("MW")

axes[0, 1].scatter(df["humidity"], df["demand_ua_adj"], alpha=0.15, s=10)
axes[0, 1].set_title("Humidity → Demand")
axes[0, 1].set_xlabel("%")

axes[0, 2].scatter(df["precipitation"], df["demand_ua_adj"], alpha=0.15, s=10)
axes[0, 2].set_title("Precipitation → Demand")
axes[0, 2].set_xlabel("mm")

axes[0, 3].scatter(df["snowfall"], df["demand_ua_adj"], alpha=0.15, s=10)
axes[0, 3].set_title("Snowfall → Demand")
axes[0, 3].set_xlabel("mm")

# -------------------------
# Row 2 — Generation
# -------------------------
axes[1, 0].scatter(df["wind_speed"], df["generation_ua_adj"], alpha=0.15, s=10)
axes[1, 0].set_title("Wind speed → Generation")
axes[1, 0].set_xlabel("m/s")
axes[1, 0].set_ylabel("MW")

axes[1, 1].scatter(df["cloud_cover"], df["generation_ua_adj"], alpha=0.15, s=10)
axes[1, 1].set_title("Cloud cover → Generation")
axes[1, 1].set_xlabel("%")

axes[1, 2].scatter(df["wind_gusts"], df["generation_ua_adj"].astype(int), alpha=0.1)
axes[1, 2].set_title("Wind gusts → Generation")
axes[1, 2].set_xlabel("m/s")

axes[1, 3].scatter(df["surface_pressure"], df["generation_ua_adj"].astype(int), alpha=0.1)
axes[1, 3].set_title("Pressure → Generation")
axes[1, 3].set_xlabel("hPa")

for ax in axes.flat:
    ax.grid(alpha=0.3)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()
